## LAB Assignment
Please finish the **Exercise** and answer **Questions**.
### Exercise (100 Points)
In this lab, our goal is to write a program to segment different objects using the **GMM and EM** algorithm. We also use <u>*k-means* clustering algorithm to initialize the parameters</u> of GMM. The following steps should be implemented to achieve such a goal:

1. Load image
2. Initialize parameters of GMM using K-means
3. Implement the EM algorithm for GMM
4. Display result

#### Load Image
What you should do is to implement Z-score normalization in `load()`:

In [1]:
# Dependency
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.cluster import KMeans
import tqdm

from PIL import Image

COLORS = [
    (255, 0, 0),   # red
    (0, 255, 0),  # green
    (0, 0, 255),   # blue
    (255, 255, 0), # yellow
    (255, 0, 255), # magenta
]

In [10]:
import cv2

def loadImage(image_path):
    image = np.array(cv2.imread(image_path))
    h, w, c = image.shape
    image = image.reshape((h*w, c))

    _mean = np.mean(image, axis = 0)
    _std = np.std(image, axis = 0)
    # TODO: please normalize image_pixl using Z-score
    normed_img = (image - _mean)/_std
        
    return h, w, c, normed_img


In [11]:
def kmeans(n_cluster, image_pixl):
    kmeans = KMeans(n_clusters=n_cluster)# instantiate a K-means
    labels = kmeans.fit_predict(image_pixl)# fit and get clustering result
    initial_mus = kmeans.cluster_centers_# get centroids
    initial_priors, initial_covs = [], []
    #Followings are for initialization:
    for i in range(n_cluster):
        datas = image_pixl[labels == i, ...].T
        initial_covs.append(np.cov(datas))
        initial_priors.append(datas.shape[1] / len(labels))
        
        
    return initial_mus, initial_priors, initial_covs

In [12]:
class GMM:
    def __init__(self, ncomp, initial_mus, initial_covs, initial_priors):
        """
        :param ncomp:           the number of clusters
        :param initial_mus:     initial means
        :param initial_covs:    initial covariance matrices
        :param initial_priors:  initial mixing coefficients
        """
        self.ncomp = ncomp
        self.mus = np.asarray(initial_mus)
        self.covs = np.asarray(initial_covs)
        self.priors = np.asarray(initial_priors)

    def inference(self, datas:np.ndarray):
        """
        E-step
        :param datas:   original data
        :return:        posterior probability (gamma) and log likelihood
        """
        probs = []
        self.mus = datas[np.random.choice(datas.shape[0], 5)]
        self.covs = np.full(fill_value = 0.1, shape = (self.ncomp, 3, 3))
        self.priors = np.repeat(1/self.ncomp, self.ncomp)
        for i in range(self.ncomp):
            mu, cov, prior = self.mus[i, :], self.covs[i, :, :], self.priors[i]
            # 出现在 cls = i 的概率
            prob = prior * multivariate_normal.pdf(
                datas, mean=mu, cov=cov, allow_singular=True
            )
            probs.append(np.expand_dims(prob, -1))
        
        # 生成一个 N * n_cls 的矩阵
        preds = np.concatenate(probs, axis=1)

        # TODO: calc log likelihood
        log_likelihood = np.sum(np.log(np.sum(preds, axis=1)))

        # TODO: calc gamma
        gamma = np.ndarray((datas.shape[0], self.ncomp))
        summ = np.sum(preds, axis=1)
        for i in range(0, datas.shape[0]):
            for j in range(0, self.ncomp):
                gamma[i, j] = (self.priors[j] * preds[i, j])/summ[i]


        return gamma, log_likelihood

    def update(self, datas:np.ndarray, gamma):
        """
        M-step
        :param datas:   original data
        :param gamma:    gamma
        :return:
        """
        new_mus, new_covs, new_priors = [], [], []
        labels = np.argmax(gamma, axis=1)
        soft_counts = np.sum(gamma, axis=0)
        
        (N, dimension) = datas.shape
        
        
        for i in range(self.ncomp):
            # TODO: calc mu
            X_ks:np.ndarray = datas[labels == i]
            N_i = X_ks.shape[0]
            if(N_i == 0):
                continue
            
            new_mu = np.zeros((dimension, ))
            for n in range(N):
                new_mu += gamma[n, i]*datas[n]
                
            
            new_mu /= N_i
                
            new_mus.append(new_mu)
            # TODO: calc cov
            new_cov = np.ndarray((dimension, dimension))
            for n in range(N):
                new_cov += gamma[n, i]*((datas[n] - new_mus[i]) * (datas[n] - new_mus[i]).T)
            
            
            new_covs.append(new_cov)

            # TODO: calc mixing coefficients
            new_prior = N_i/N
            new_priors.append(new_prior)
            

        self.mus = np.asarray(new_mus)
        self.covs = np.asarray(new_covs)
        self.priors = np.asarray(new_priors)
        

    def fit(self, data, iteration):
        prev_log_liklihood = None

        bar = tqdm.tqdm(total=iteration)
        for i in range(iteration):
            gamma, log_likelihood = self.inference(data)
            self.update(data, gamma)
            if prev_log_liklihood is not None and abs(log_likelihood - prev_log_liklihood) < 1e-10:
                break
            prev_log_likelihood = log_likelihood

            bar.update()
            bar.set_postfix({"log likelihood": log_likelihood})         
     

#### Display
We use `matplotlib` to display what we segment, you can check the code in `visualize()`

In [13]:
from PIL import Image
import matplotlib.pyplot as plt


def visualize(gmm, image, ncomp, ih, iw):
    beliefs, log_likelihood = gmm.inference(image)
    map_beliefs = np.reshape(beliefs, (ih, iw, ncomp))
    segmented_map = np.zeros((ih, iw, 3))
    for i in range(ih):
        for j in range(iw):
            hard_belief = np.argmax(map_beliefs[i, j, :])
            segmented_map[i, j, :] = np.asarray(COLORS[hard_belief]) / 255.0
            
            
    plt.imshow(segmented_map)
    plt.show()
    
    

h, w, c, image = loadImage("/Users/mulikas/Codes/CS405 ML/Lab/Lab12.GMM clustering 2/data/original/sample.png")
n = image.shape[0]

n_cls = 3

initial_mus, initial_priors, initial_covs = kmeans(n_cls, image)

gmm = GMM(n_cls, initial_mus, initial_covs, initial_priors)
gmm.fit(image, 100)
visualize(gmm, image, n_cls, h, w)

KeyboardInterrupt: 

#### sample Result
<img src="images/image-20220804223008133.png" alt="image-20220804223008133" style="zoom:67%;" />
<img src="images/image-20220804222915979.png" alt="image-20220804222915979" style="zoom: 67%;" />

### Questions(3 points)
1. What are the strengths of GMM; when does it perform well?
2. What are the weaknesses of GMM; when does it perform poorly?
3. What makes GMM a good candidate for the clustering problem, if you have enough knowledge about the data?

1. 优点: GMM的优点是投影后样本点不是得到一个确定的分类标记，而是得到每个类的概率，这是一个重要信息。GMM不仅可以用在聚类上，也可以用在概率密度估计上。
2. 缺点:当每个混合模型没有足够多的点时，估算协方差变得困难起来，同时算法会发散并且找具有无穷大似然函数值的解，除非人为地对协方差进行正则化。GMM每一步迭代的计算量比较大，大于k-means。GMM的求解办法基于EM算法，因此有可能陷入局部极值，这和初始值的选取十分相关了。
3. 聚类的同时需要额外获得概率密度，样本可分性明显。